## Producer 2

**Description**: 
Gets the rating csv's from the datasink folder and sends them to the specified framework kafka or rabbitmq.

In [ ]:
import json
import time
from helper_file import Producer_1
import pandas as pd
import os

topic = "top_ratings"
path = "src/datasink/"
hz = 1/10

producer = Producer_1(framework='kafka', host_name="broker1", port=9093)
# producer = Producer_1(framework='rabbitmq', host_name="rabbitmq1", port=5672)

while True:
    for file in os.listdir(path):
        if file.endswith(".csv"):
            df = pd.read_csv(path+file)
            message = df.to_json()
            producer.produce(topic, message)
            print(f"File: {file} published")
            os.remove(path+file)
    time.sleep(1/hz)